# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [146]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [147]:
city_data_df = city_csv_df[["name", "sys.country", "main.humidity", "coord.lon", "coord.lat"]]
city_data_df.head()

,name,sys.country,main.humidity,coord.lon,coord.lat
0,Hobart,AU,74.0,147.3294,-42.8794
1,Vaini,TO,88.0,-175.2000,-21.2000
2,Lata,PT,36.0,-8.3327,40.1629
3,Butaritari,KI,77.0,172.7902,3.0707
4,Saint-Pierre,RE,67.0,55.4781,-21.3393


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [148]:
rename_city_data_df = city_data_df.rename(columns={"name":"city", "sys.country": "Country", "main.humidity": "Humidity"})
rename_city_data_df

,city,Country,Humidity,coord.lon,coord.lat
0,Hobart,AU,74.0,147.3294,-42.8794
1,Vaini,TO,88.0,-175.2000,-21.2000
2,Lata,PT,36.0,-8.3327,40.1629
3,Butaritari,KI,77.0,172.7902,3.0707
4,Saint-Pierre,RE,67.0,55.4781,-21.3393
...,...,...,...,...,...
608,Tumaco,CO,87.0,-78.8156,1.7986
609,NaN,NaN,NaN,NaN,NaN
610,Panama City,PA,82.0,-79.5197,8.9936
611,Waingapu,ID,75.0,120.2641,-9.6567


In [149]:
clean_rnull_df = rename_city_data_df.dropna()
clean_rnull_df

,city,Country,Humidity,coord.lon,coord.lat
0,Hobart,AU,74.0,147.3294,-42.8794
1,Vaini,TO,88.0,-175.2000,-21.2000
2,Lata,PT,36.0,-8.3327,40.1629
3,Butaritari,KI,77.0,172.7902,3.0707
4,Saint-Pierre,RE,67.0,55.4781,-21.3393
...,...,...,...,...,...
607,Kamenka,RU,49.0,42.7678,51.3223
608,Tumaco,CO,87.0,-78.8156,1.7986
610,Panama City,PA,82.0,-79.5197,8.9936
611,Waingapu,ID,75.0,120.2641,-9.6567


In [150]:
locations = clean_rnull_df[["coord.lat", "coord.lon"]]
weights = clean_rnull_df["Humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [151]:
rename_city_data_df.iloc[10, :]

city         Santa Maria
Country               BR
Humidity           100.0
coord.lon       -53.8069
coord.lat       -29.6842
Name: 10, dtype: object

In [183]:
row_df = rename_city_data_df.iloc[0:10, :]
row_df

,city,Country,Humidity,coord.lon,coord.lat
0,Hobart,AU,74.0,147.3294,-42.8794
1,Vaini,TO,88.0,-175.2000,-21.2000
2,Lata,PT,36.0,-8.3327,40.1629
3,Butaritari,KI,77.0,172.7902,3.0707
4,Saint-Pierre,RE,67.0,55.4781,-21.3393
5,Bluff,NZ,95.0,168.3333,-46.6000
6,NaN,NaN,NaN,NaN,NaN
7,Avarua,CK,73.0,-159.7750,-21.2078
8,Preobrazheniye,RU,93.0,133.9064,42.9019
9,Hamilton,US,79.0,-84.5333,39.1834


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [171]:
row_df.count()

city         9
Country      9
Humidity     9
coord.lon    9
coord.lat    9
dtype: int64

In [172]:
clean_data_df = row_df.dropna()
clean_data_df.dtypes

city          object
Country       object
Humidity     float64
coord.lon    float64
coord.lat    float64
dtype: object

In [176]:
locations = clean_data_df[["coord.lat", "coord.lon"]]
weights = clean_data_df["Humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [156]:
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [187]:
new_df = clean_rnull_df.drop(columns=["Humidity", "coord.lat", "coord.lon"])
new_df.head()

,city,Country
0,Hobart,AU
1,Vaini,TO
2,Lata,PT
3,Butaritari,KI
4,Saint-Pierre,RE


In [ ]:
new_df

In [177]:
clean_data_df["Hotel Name"] = ""
clean_data_df["Hotel Address"] = ""
clean_data_df.head()

<ipython-input-177-4543857446fb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data_df["Hotel Name"] = ""
<ipython-input-177-4543857446fb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data_df["Hotel Address"] = ""


,city,Country,Humidity,coord.lon,coord.lat,Hotel Name,Hotel Address
0,Hobart,AU,74.0,147.3294,-42.8794,,
1,Vaini,TO,88.0,-175.2000,-21.2000,,
2,Lata,PT,36.0,-8.3327,40.1629,,
3,Butaritari,KI,77.0,172.7902,3.0707,,
4,Saint-Pierre,RE,67.0,55.4781,-21.3393,,


In [181]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "international hotel",
    "api_keys": g_key
}
# use the lat/lng
for index, row in clean_data_df.iterrows():
    lat = row["coord.lat"]
    lng = row["coord.lon"]
    params["locations"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    response = requests.get(base_url, params=params)
    name_address = response.json()
    name_address
    try:
        clean_data_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        clean_data_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (keyError, IndexError):
        print("Missing field/results...skipping")

NameError: name 'keyError' is not defined

In [166]:
len(name_address)

4

In [167]:
type(name_address)

dict

In [168]:
clean_data_df.to_csv("hotel_output.csv")
clean_data_df.head()

,index,city,Country,Humidity,coord.lon,coord.lat,Hotel Name,Hotel Address
0,1,Vaini,TO,88.0,-175.2000,-21.2000,,
1,2,Lata,PT,36.0,-8.3327,40.1629,,
2,3,Butaritari,KI,77.0,172.7902,3.0707,,
3,4,Saint-Pierre,RE,67.0,55.4781,-21.3393,,
4,5,Bluff,NZ,95.0,168.3333,-46.6000,,


In [180]:
clean_data_df["results"]

KeyError: 'results'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
